In [1]:
from PIL import Image
import torch
from torchvision import transforms
from torch import nn
from torchvision.models import resnet34, ResNet34_Weights
import numpy as np
import os
import pandas as pd

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = (224, 224)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [3]:
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [4]:
model = resnet34(weights=ResNet34_Weights.DEFAULT)
model.fc = nn.Identity()
model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
def extract_features(image_path):
    img = Image.open(image_path).convert('RGB')
    img = transform(img)

    img = img.unsqueeze(0).to(device)

    with torch.no_grad():
        features = model(img)

    return features.squeeze().cpu().numpy()

In [6]:
feature_list = []
image_names = []
for image_path in os.listdir("faces"):
    if image_path.endswith(".jpg"):
        features = extract_features(os.path.join("faces", image_path))
        feature_list.append(features)
        image_names.append(image_path)

In [7]:
features = np.array(feature_list)

In [8]:
features

array([[2.7266729e-01, 1.8130443e-01, 5.2261305e-01, ..., 4.4510755e-01,
        1.0084591e-02, 2.2795089e-01],
       [2.7974856e-01, 1.9155865e-03, 1.1510092e-01, ..., 5.6593293e-01,
        2.3556963e-02, 0.0000000e+00],
       [1.9152623e-01, 3.0920136e-01, 1.4449698e-01, ..., 8.7487167e-01,
        3.2340158e-02, 7.6012433e-02],
       ...,
       [1.2337389e+00, 2.5335188e+00, 1.0338317e+00, ..., 1.3521023e+00,
        4.3273956e-01, 5.8018929e-01],
       [1.4665031e-01, 5.9854084e-01, 7.2720951e-01, ..., 1.0651809e+00,
        2.3365585e-02, 9.0102799e-02],
       [9.0118498e-02, 7.6999116e-01, 3.8016029e-02, ..., 7.0997262e-01,
        3.0191916e-01, 2.2381283e-02]], dtype=float32)

In [9]:
df = pd.DataFrame(features)
df.insert(0, 'Image Name', image_names)
df.to_csv('features.csv', index=False)

In [10]:
df = pd.read_csv('features.csv')
df

,Image Name,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,10002702_1960-11-09_2012.jpg,0.272667,0.181304,0.522613,0.985274,0.107050,0.359303,0.275444,0.677732,0.263400,...,0.000000,0.181292,0.383862,0.020063,0.129796,0.336707,0.028761,0.445108,0.010085,0.227951
1,1002001_1944-05-01_1995.jpg,0.279749,0.001916,0.115101,0.893894,0.040134,0.333267,0.071751,0.759813,0.170035,...,0.006483,0.048637,0.739707,0.014779,0.554291,0.208090,0.462767,0.565933,0.023557,0.000000
2,1004604_1981-11-27_2012.jpg,0.191526,0.309201,0.144497,1.417195,0.191475,0.046141,0.252428,0.148858,0.103499,...,0.007329,0.002504,0.347986,0.126279,0.739436,0.297104,0.387431,0.874872,0.032340,0.076012
3,10049200_1891-09-16_1958.jpg,0.382820,0.079980,0.591635,0.931270,0.348299,0.317523,0.126595,0.221747,0.059046,...,0.040913,0.001872,1.560098,0.123703,0.385894,0.007108,0.017871,1.083520,0.023397,0.091238
4,10064003_1986-01-04_2009.jpg,0.865130,0.361720,0.539288,1.447548,0.425298,0.970224,0.440898,0.828271,0.076451,...,0.101955,0.025929,0.740600,0.087209,0.401073,0.026632,0.077148,0.935509,0.135685,0.063321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974,9975102_1964-09-12_2013.jpg,0.631586,0.713522,0.525309,1.352313,0.651255,0.496266,0.179776,0.967858,0.305912,...,0.014055,0.230134,0.291623,0.176886,0.095216,0.651918,0.271787,0.542943,0.000000,0.304776
2975,999202_1978-09-15_2006.jpg,0.149437,0.326232,0.527751,0.626464,0.089703,0.717217,0.046903,0.415258,0.001581,...,0.012237,0.069713,1.871960,0.029520,1.648195,0.103237,0.084084,0.679717,0.106744,0.358840
2976,9994102_1992-03-13_2013.jpg,1.233739,2.533519,1.033832,3.189962,1.146127,3.177276,1.651978,1.291490,1.392432,...,0.406986,0.022475,0.274133,0.453498,0.808227,0.931516,1.671831,1.352102,0.432740,0.580189
2977,9996800_1941-01-28_1966.jpg,0.146650,0.598541,0.727209,1.537440,0.000000,0.574576,0.128012,0.287720,0.054909,...,0.151382,0.103588,1.688827,0.037238,0.604387,0.063688,0.133424,1.065181,0.023366,0.090103
